In [1]:
import sys
sys.argv.append('--JOB_NAME')
sys.argv.append('address-cleaning')

sys.argv.append('--cleaned_addresses_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-refined-zone/housing/repairs-dlo/cleanedAddresses')

sys.argv.append('--source_catalog_database')
sys.argv.append('dataplatform-stg-raw-zone-database')

sys.argv.append('--source_catalog_table')
sys.argv.append('housing_repairs_dlo')

sys.argv.append('--source_address_column_header')
sys.argv.append('address_of_repair')

sys.argv.append('--source_postcode_column_header')
sys.argv.append('postcode_of_property')



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, trim, when, max
import pyspark.sql.functions as F
from awsglue.dynamicframe import DynamicFrame

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default
    
def getLatestPartitions(dfa):
   dfa = dfa.where(col('import_year') == dfa.select(max('import_year')).first()[0])
   dfa = dfa.where(col('import_month') == dfa.select(max('import_month')).first()[0])
   dfa = dfa.where(col('import_day') == dfa.select(max('import_day')).first()[0])
   return dfa

## write into the log file with:
## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

source_catalog_database = get_glue_env_var('source_catalog_database', '')
source_catalog_table = get_glue_env_var('source_catalog_table', '')
source_address_column_header = get_glue_env_var('source_address_column_header', '')
source_postcode_column_header = get_glue_env_var('source_postcode_column_header', '')
cleaned_addresses_s3_bucket_target = get_glue_env_var('cleaned_addresses_s3_bucket_target', '')

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
logger.info('Fetch Source Data')
source_dataset = glueContext.create_dynamic_frame.from_catalog(
    database=source_catalog_database,
    table_name=source_catalog_table,
    transformation_ctx='source_dataset',
)

df = source_dataset.toDF()
source_dataset.printSchema()

df = getLatestPartitions(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`import_year`' given input columns: [];;\n'Aggregate [max('import_year) AS max(import_year)#2]\n+- LogicalRDD false\n"
Traceback (most recent call last):
  File "<stdin>", line 21, in getLatestPartitions
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1320, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "cannot resolve '`import_year`' given input columns: [];;\n'Aggregate [max('import_year) AS max(import_year)#2]\n+- LogicalRDD false\n"



In [ ]:
tmp = df[[
    'name_of_resident',
    'address_of_repair',
    'postcode_of_property',
    'import_datetime',
    'import_date',
    'import_timestamp',
    'import_year'
]]

tmp.show()


In [ ]:
logger.info('adding new column')
df = df.withColumn('address', F.col(source_address_column_header))

logger.info('extract postcode into a new column')
df = df.withColumn('postcode', F.regexp_extract(F.col('address'), '([A-Za-z][A-Ha-hJ-Yj-y]?[0-9][A-Za-z0-9]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})', 1))

logger.info('remove postcode from address')
df = df.withColumn('address', F.regexp_replace(F.col('address'), '([A-Za-z][A-Ha-hJ-Yj-y]?[0-9][A-Za-z0-9]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})', ''))

logger.info('populate empty postcode with postcode from the other PC column')
df = df.withColumn("postcode", \
       F.when(F.col("postcode")=="" ,None) \
          .otherwise(F.col("postcode")))

if source_postcode_column_header:
    df = df.withColumn("postcode", F.coalesce(F.col('postcode'),F.col(source_postcode_column_header)))

logger.info('postcode formatting')
df = df.withColumn("postcode", F.upper(F.col("postcode")))
df = df.withColumn("postcode_nospace", F.regexp_replace(F.col("postcode"), " +", ""))
df = df.withColumn("postcode_length", F.length(F.col("postcode_nospace"))) 
df = df.withColumn("postcode_start", F.expr("substring(postcode_nospace, 1, postcode_length -3)"))
df = df.withColumn("postcode_end", F.expr("substring(postcode_nospace, -3, 3)"))
df = df.withColumn("postcode", F.concat_ws(" ", "postcode_start", "postcode_end"))

logger.info('address line formatting - remove commas and extra spaces')
df = df.withColumn("address", F.upper(F.col("address")))
df = df.withColumn("address", F.regexp_replace(F.col("address"), ",", ""))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " +", " "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " ?- ?\z", ""))


logger.info('address line formatting - remove LONDON at the end (dont do this for out of London matching)')
df = df.withColumn("address", F.trim(F.col("address")))
df = df.withColumn("address_length", F.length(F.col("address"))) 
df = df.withColumn("address", \
       F.when(F.col("address").endswith(" LONDON"), F.expr("substring(address, 1, address_length -7)")) \
          .otherwise(F.col("address")))
                   
logger.info('address line formatting - remove HACKNEY at the end (dont necessarily this for out of borough matching)')
df = df.withColumn("address", F.trim(F.col("address")))
df = df.withColumn("address_length", F.length(F.col("address"))) 
df = df.withColumn("address", \
       F.when(F.col("address").endswith(" HACKNEY"), F.expr("substring(address, 1, address_length -8)")) \
          .otherwise(F.col("address")))

logger.info('address line formatting - dashes between numbers: remove extra spaces')
df = df.withColumn("address", F.regexp_replace(F.col("address"), '(\\d+) ?- ?(\\d+)', '$1-$2'))

logger.info('deal with abbreviations')

logger.info('for \'street\': we only replace st if it is at the end of the string, if not there is a risk of confusion with saint')
df = df.withColumn("address", F.regexp_replace(F.col("address"), " ST.?\z", " STREET"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " RD.? ", " ROAD "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " RD.?\z", " ROAD"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " AVE ", " AVENUE "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " AVE\z", " AVENUE"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " HSE ", " HOUSE "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " HSE\z", " HOUSE"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " CT.? ", " COURT "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " CT.?\z", " COURT"))

# df = df.withColumn("address", F.regexp_replace(F.col("address"), " ST.? ", " SAINT "))

df = df.withColumnRenamed("address", "concatenated_string_to_match")

logger.info('create a unique ID')
df = df.withColumn("prinx", F.monotonically_increasing_id()).repartition(1)

logger.info('write into parquet')
cleanedDataframe = DynamicFrame.fromDF(df, glueContext, "cleanedDataframe")

parquetData = glueContext.write_dynamic_frame.from_options(
    frame=cleanedDataframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": cleaned_addresses_s3_bucket_target, "partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="parquetData")

job.commit()